In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import newton

def calculate_irr(cash_flows, initial_guess=0.1):
    """Calculates the Internal Rate of Return (IRR) for a series of cash flows.

    Args:
        cash_flows (list or pd.Series): A list or Pandas Series representing the cash flows.
                                          The first element is typically the initial investment (negative).
        initial_guess (float): Initial guess for the IRR calculation.

    Returns:
        float: The IRR as a decimal (e.g., 0.15 for 15%).  Returns NaN if IRR can't be calculated.
    """

    if not isinstance(cash_flows, (list, pd.Series, np.ndarray)):
        raise TypeError("Cash flows must be a list, pandas Series, or numpy array.")

    cash_flows = np.array(cash_flows)

    if cash_flows.size < 2:
        return np.nan  # Need at least an initial investment and one return.

    # Use Newton's method to find the root (IRR)
    try:
        irr = newton(lambda r: np.sum(cash_flows / (1 + r)**np.arange(len(cash_flows))), initial_guess)
        return irr
    except RuntimeError:
        return np.nan #Indicates that the IRR calculation failed to converge

def elasticity_irr(df, cf_row, gp_equity_col, lp_equity_col, first_cf_col, last_cf_col, irr_col):
  """
  Calculates the elasticity of GP and LP IRR to a percentage increase in cash flows,
  given the data and row/column locations.

  Args:
      df (pd.DataFrame): DataFrame containing the cash flow data and parameters.
      cf_row (int): Row index in df of the cash flow row.
      gp_equity_col (str): Column label of the GP equity investment.
      lp_equity_col (str): Column label of the LP equity investment.
      first_cf_col (str): Column label of the first cash flow column.
      last_cf_col (str): Column label of the last cash flow column.
      irr_col (str): Column label to store original IRR.

  Returns:
      pd.DataFrame: The DataFrame with added rows for new IRR values for base, base +10%, base +20%
  """

  # Calculate base case IRR
  cash_flows_base = [-df.loc[df.index[0], gp_equity_col] - df.loc[df.index[0], lp_equity_col]] + df.loc[cf_row, first_cf_col:last_cf_col].tolist()
  df.loc[df.index[-3], irr_col] = calculate_irr(cash_flows_base)

  # Increase CF by 10% and calculate new IRR
  df.loc[df.index[-2], :] = df.loc[cf_row, :]
  cash_flows_plus_10 = [-df.loc[df.index[-2], gp_equity_col] - df.loc[df.index[-2], lp_equity_col]] + (df.loc[cf_row, first_cf_col:last_cf_col] * 1.1).tolist()
  df.loc[df.index[-2], irr_col] = calculate_irr(cash_flows_plus_10)

   # Increase CF by 20% and calculate new IRR
  df.loc[df.index[-1], :] = df.loc[cf_row, :]
  cash_flows_plus_20 = [-df.loc[df.index[-1], gp_equity_col] - df.loc[df.index[-1], lp_equity_col]] + (df.loc[cf_row, first_cf_col:last_cf_col] * 1.2).tolist()
  df.loc[df.index[-1], irr_col] = calculate_irr(cash_flows_plus_20)

  return df


# Load data into a Pandas DataFrame
data = {
    'RETURNS': ['GP Equity', 'GP Equity - Step up in Land Basis taken as Fee vs Promote', 'LP Equity', 'Total', 'Cash Flow Available for Distribution'],
    'C': [715232.76, 411049, 7056604.92, 7771837.68, 100000],
    'D': [0.09203, None, 0.90797, 1.0, 100],
    'Initial': ['Totals', 'Initial', 1.862645149230957E-8, 411049, 100],
    'E': ['Preferred Return', 1.862645149230957E-8, 1, 'EOMONTH(D15,1)', 100],
    'F': ['First Tier', 1.862645149230957E-8, 2, 'EOMONTH(E15,1)', 100],
    'G': ['Second Tier', 1.862645149230957E-8, 3, 'EOMONTH(F15,1)', 100],
    'H': ['Third Tier', 1.862645149230957E-8, 4, 411172, 100],
    'I': [None, 1.862645149230957E-8, 5, 411203, 100],
    'J': [None, 1.862645149230957E-8, 6, 411233, 100],
    'K': [None, 1.862645149230957E-8, 7, 411264, 100],
    'L': [None, 1.862645149230957E-8, 8, 411295, 100],
    'M': [None, None, 9, 411323, 100],
    'N': [None, None, 10, 411354, 100],
    'O': [None, None, 11, 411384, 100],
    'P': [None, None, 12, 411415, 100],
    'Q': [None, None, 13, 411445, 100],
    'R': [None, None, 14, 411476, 100],
    'S': [None, None, 15, 411507, 100],
    'T': [None, None, 16, 411537, 100],
    'U': [None, None, 17, 411568, 100],
    'V': [None, None, 18, 411598, 100],
    'W': [None, None, 19, 411629, 100],
    'X': [None, None, 20, 411660, 100],
    'Y': [None, None, 21, 411688, 100],
    'Z': [None, None, 22, 411719, 100],
    'AA': [None, None, 23, 411749, 100],
    'AB': [None, None, 24, 411780, 100],
    'AC': [None, None, 25, 411810, 100],
    'AD': [None, None, 26, 411841, 100],
    'AE': [None, None, 27, 411872, 100],
    'AF': [None, None, 28, 411902, 100],
    'AG': [None, None, 29, 411933, 100],
    'AH': [None, None, 30, 411963, 100],
    'AI': [None, None, 31, 411994, 100],
    'AJ': [None, None, 32, 412025, 100],
    'AK': [None, None, 33, 412054, 100],
    'AL': [None, None, 34, 412085, 100],
    'AM': [None, None, 35, 412115, 100],
    'AN': [None, None, 36, 412146, 100],
    'AO': [None, None, 37, 412176, 100],
    'AP': [None, None, 38, 412207, 100],
    'AQ': [None, None, 39, 412238, 100],
    'AR': [None, None, 40, 412268, 100],
    'AS': [None, None, 41, 412299, 100],
    'AT': [None, None, 42, 412329, 100],
    'AU': [None, None, 43, 412360, 100],
    'AV': [None, None, 44, 412391, 100],
    'AW': [None, None, 45, 412419, 100],
    'AX': [None, None, 46, 412450, 100],
    'AY': [None, None, 47, 412480, 100],
    'AZ': [None, None, 48, 412511, 100],
    'BA': [None, None, 49, 412541, 100],
    'BB': [None, None, 50, 412572, 100],
    'BC': [None, None, 51, 412603, 100],
    'BD': [None, None, 52, 412633, 100],
    'BE': [None, None, 53, 412664, 100],
    'BF': [None, None, 54, 412694, 100],
    'BG': [None, None, 55, 412725, 100],
    'BH': [None, None, 56, 412756, 100],
    'BI': [None, None, 57, 412784, 100],
    'BJ': [None, None, 58, 412815, 100],
    'BK': [None, None, 59, 412845, 100],
    'BL': [None, None, 60, 412876, 100],
    'GP IRR':[None,None, None, None, None],
    'LP IRR':[None,None, None, None, None]

}

df = pd.DataFrame(data)
df = df.set_index('RETURNS')

# Ensure columns exist (handling potential missing columns from data)
first_month_col = 'D'  # First month cash flow column
last_month_col  = 'AZ' #Last month cash flow column
gp_equity_keyword = 'GP Equity' # row to search for GP equity
lp_equity_keyword = 'LP Equity' #row to search for LP equity
cf_row_keyword = 'Cash Flow Available for Distribution' #Row to start from cash flow
irr_col_gp = 'GP IRR'  #Column label for GP IRR
irr_col_lp = 'LP IRR'   #Column label for LP IRR
new_rows = ['Base Case', 'Base Case + 10%', 'Base Case + 20%']
df = df.reindex(df.index.tolist() + new_rows)

# Find the  rows based on keywords
gp_equity_row = df.index.get_loc(gp_equity_keyword)  #Find the index by name
lp_equity_row = df.index.get_loc(lp_equity_keyword)
cf_row = cf_row_keyword #df.index.get_loc(cf_row_keyword)


# Find the GP and LP equity investment from the sheet.

gp_equity = df.loc[gp_equity_keyword, 'C']
lp_equity = df.loc[lp_equity_keyword, 'C']
df.loc['Base Case', 'GP Equity'] = gp_equity
df.loc['Base Case', 'LP Equity'] = lp_equity

# Get list of cash flows per period

first_month_col = 'D'  # First month cash flow column
last_month_col  = 'AZ' #Last month cash flow column
cashflow_row = 'Cash Flow Available for Distribution'
cash_flow = df.loc[cashflow_row, first_month_col:last_month_col].astype(float).fillna(0)

first_cf_col = first_month_col
last_cf_col = last_month_col

#Now build the IRR for the GP

df = elasticity_irr(df, cashflow_row, 'C', 'C', first_cf_col, last_cf_col, 'GP IRR')


print(df[['GP IRR', 'LP IRR']])

                                                   GP IRR LP IRR
RETURNS                                                         
GP Equity                                            None   None
GP Equity - Step up in Land Basis taken as Fee ...   None   None
LP Equity                                            None   None
Total                                                None   None
Cash Flow Available for Distribution                 None   None
Base Case                                             NaN    NaN
Base Case + 10%                                       NaN   None
Base Case + 20%                                       NaN   None


C:\Users\i7714\AppData\Local\Temp\ipykernel_22500\1204259235.py:27: RuntimeWarning: overflow encountered in power
  irr = newton(lambda r: np.sum(cash_flows / (1 + r)**np.arange(len(cash_flows))), initial_guess)
C:\Users\i7714\AppData\Local\Temp\ipykernel_22500\1204259235.py:27: RuntimeWarning: overflow encountered in power
  irr = newton(lambda r: np.sum(cash_flows / (1 + r)**np.arange(len(cash_flows))), initial_guess)
C:\Users\i7714\AppData\Local\Temp\ipykernel_22500\1204259235.py:27: RuntimeWarning: overflow encountered in power
  irr = newton(lambda r: np.sum(cash_flows / (1 + r)**np.arange(len(cash_flows))), initial_guess)
